In [1]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [2]:
from langchain_community.llms import Tongyi

import getpass 
import os 

def _set_env(var: str): 
    if not os.environ.get(var): 
        os.environ[var] = getpass.getpass(f"{var}: ") 

# api key: sk-a31a563bce7146bf8c8ab93ad5fea537
_set_env("DASHSCOPE_API_KEY")

llm = Tongyi()

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

DASHSCOPE_API_KEY:  ········


In [3]:
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [5]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User:  你好


User: What do you know about LangGraph?


AttributeError: 'str' object has no attribute 'content'